In [5]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.exceptions import UndefinedMetricWarning

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPRegressor

Load final boards classification dataset

In [2]:
# This dataset contains values representing a final tic-tac-toe board and the game's winner
final_data = np.loadtxt('tictac_final.txt')
input_features = final_data[:,:9]
output_labels = final_data[:,9:]

input_features

array([[ 1.,  1.,  1., ...,  1., -1., -1.],
       [ 1.,  1.,  1., ..., -1.,  1., -1.],
       [ 1.,  1.,  1., ..., -1., -1.,  1.],
       ...,
       [-1.,  1., -1., ...,  1., -1.,  1.],
       [-1.,  1., -1., ...,  1., -1.,  1.],
       [-1., -1.,  1., ..., -1.,  1.,  1.]])

In [3]:
output_labels # The winning player, represented as 1 or -1

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
      

Create train-test split

In [5]:
train, test = train_test_split(final_data, test_size=0.20)

x_train = train[:,:9]
x_test = test[:,:9]
y_train = train[:,9:]
y_test = test[:,9:]

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(766, 9)
(192, 9)
(766, 1)
(192, 1)


Linear SVM classifier on final boards dataset

In [7]:
clf = SVC(kernel='linear', max_iter=1000)
clf.fit(x_train, np.ravel(y_train, order='C'))

y_pred = clf.predict(x_test)

metrics.accuracy_score(y_test, y_pred)

0.9895833333333334

In [9]:
# Cross validation

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

accuracies = cross_val_score(clf, x_train, y_train, cv=cv, scoring="accuracy", n_jobs=-1)
acc_svm = np.mean(accuracies)
acc_svm

0.9817156527682843

In [10]:
# Confusion matrix

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred, normalize='true')

print(cm)

# Confusion matrix accuracy

correct = cm[0, 0] + cm[1, 1]
total = correct + cm[0, 1] + cm[1, 0]

cm_accuracy = correct / total

cm_accuracy

[[0.97142857 0.02857143]
 [0.         1.        ]]


0.9857142857142858

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       1.00      0.97      0.99        70
         1.0       0.98      1.00      0.99       122

    accuracy                           0.99       192
   macro avg       0.99      0.99      0.99       192
weighted avg       0.99      0.99      0.99       192



K-Neighbors classifier on final boards

In [18]:
clf = KNeighborsClassifier()

clf.fit(x_train, np.ravel(y_train, order='C'))

y_pred = clf.predict(x_test)

print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

# Cross validation

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

accuracies = cross_val_score(clf, x_train, y_train, cv=cv, scoring="accuracy", n_jobs=-1)
acc_knn = np.mean(accuracies)
print('Cross validation score: ', acc_knn)

# Confusion matrix

cm = confusion_matrix(y_test, y_pred, normalize='true')

print('Confusion matrix: \n', cm)

# Confusion matrix accuracy

correct = cm[0, 0] + cm[1, 1]
total = correct + cm[0, 1] + cm[1, 0]

cm_accuracy = correct / total

cm_accuracy
print(classification_report(y_test, y_pred))

Accuracy:  1.0
Cross validation score:  0.9939052175894282
Confusion matrix: 
 [[1. 0.]
 [0. 1.]]
              precision    recall  f1-score   support

        -1.0       1.00      1.00      1.00        70
         1.0       1.00      1.00      1.00       122

    accuracy                           1.00       192
   macro avg       1.00      1.00      1.00       192
weighted avg       1.00      1.00      1.00       192



Multi-Layer Perceptron classifier on final boards

In [21]:
clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, activation='relu', random_state=42)

clf.fit(x_train, np.ravel(y_train, order='C'))

y_pred = clf.predict(x_test)

print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

# Cross validation

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

accuracies = cross_val_score(clf, x_train, y_train, cv=cv, scoring="accuracy", n_jobs=-1)
acc_mlp = np.mean(accuracies)
print('Cross validation score: ', acc_mlp)

# Confusion matrix

cm = confusion_matrix(y_test, y_pred, normalize='true')

print('Confusion matrix: \n', cm)

# Confusion matrix accuracy

correct = cm[0, 0] + cm[1, 1]
total = correct + cm[0, 1] + cm[1, 0]

cm_accuracy = correct / total

cm_accuracy
print(classification_report(y_test, y_pred))

Accuracy:  0.9895833333333334
Cross validation score:  0.9847573479152426
Confusion matrix: 
 [[0.97142857 0.02857143]
 [0.         1.        ]]
              precision    recall  f1-score   support

        -1.0       1.00      0.97      0.99        70
         1.0       0.98      1.00      0.99       122

    accuracy                           0.99       192
   macro avg       0.99      0.99      0.99       192
weighted avg       0.99      0.99      0.99       192



Load intermediate boards, optimal play (single-output) dataset

In [2]:
single_data = np.loadtxt('tictac_single.txt')
input_features = single_data[:,:9]
output_labels = single_data[:,9:]

print(input_features)
print(output_labels)

[[ 1. -1.  0. ...  0.  1.  0.]
 [ 1.  0.  1. ...  0. -1.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [-1. -1.  0. ...  1.  1.  0.]
 [ 0.  1.  1. ...  0.  1. -1.]
 [ 1.  1. -1. ... -1. -1.  0.]]
[[6.]
 [1.]
 [2.]
 ...
 [2.]
 [0.]
 [4.]]


Train-test split

In [3]:
train, test = train_test_split(single_data, test_size=0.20)

x_train = train[:,:9]
x_test = test[:,:9]
y_train = train[:,9:]
y_test = test[:,9:]

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5240, 9)
(1311, 9)
(5240, 1)
(1311, 1)


Linear SVM classifier on single-output dataset

In [32]:
clf = SVC(kernel='linear', max_iter=500)
with ignore_warnings(category=ConvergenceWarning):
    clf.fit(x_train, np.ravel(y_train, order='C'))

y_pred = clf.predict(x_test)

print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

# Cross validation

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

accuracies = cross_val_score(clf, x_train, y_train, cv=cv, scoring="accuracy", n_jobs=-1)
acc_svm = np.mean(accuracies)
print('Cross-validation score: ', acc_svm)

from sklearn.metrics import multilabel_confusion_matrix

# Confusion matrix

cm = multilabel_confusion_matrix(y_test, y_pred)

print('Confusion matrix: \n', cm)


Accuracy:  0.2295957284515637
Cross-validation score:  0.23874045801526714
Confusion matrix: 
 [[[ 611  406]
  [ 149  145]]

 [[ 916  235]
  [ 107   53]]

 [[ 856  256]
  [ 126   73]]

 [[1198   18]
  [  94    1]]

 [[1020   70]
  [ 197   24]]

 [[1225   11]
  [  72    3]]

 [[1176    9]
  [ 126    0]]

 [[1265    0]
  [  46    0]]

 [[1211    5]
  [  93    2]]]


In [33]:
with ignore_warnings(category=UndefinedMetricWarning):
    print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.26      0.49      0.34       294
         1.0       0.18      0.33      0.24       160
         2.0       0.22      0.37      0.28       199
         3.0       0.05      0.01      0.02        95
         4.0       0.26      0.11      0.15       221
         5.0       0.21      0.04      0.07        75
         6.0       0.00      0.00      0.00       126
         7.0       0.00      0.00      0.00        46
         8.0       0.29      0.02      0.04        95

    accuracy                           0.23      1311
   macro avg       0.16      0.15      0.13      1311
weighted avg       0.19      0.23      0.18      1311



^^ Yikes, the data is probably not well represented linearly

In [34]:
clf = SVC(kernel='rbf', max_iter=500)
with ignore_warnings(category=ConvergenceWarning):
    clf.fit(x_train, np.ravel(y_train, order='C'))

y_pred = clf.predict(x_test)

print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

# Cross validation

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

accuracies = cross_val_score(clf, x_train, y_train, cv=cv, scoring="accuracy", n_jobs=-1)
acc_svm = np.mean(accuracies)
print('Cross-validation score: ', acc_svm)

from sklearn.metrics import multilabel_confusion_matrix

# Confusion matrix

cm = multilabel_confusion_matrix(y_test, y_pred)

print('Confusion matrix: \n', cm)

Accuracy:  0.8237986270022883
Cross-validation score:  0.811704834605598
Confusion matrix: 
 [[[ 913  104]
  [  14  280]]

 [[1123   28]
  [  35  125]]

 [[1087   25]
  [  39  160]]

 [[1203   13]
  [  36   59]]

 [[1060   30]
  [  26  195]]

 [[1228    8]
  [  17   58]]

 [[1176    9]
  [  28   98]]

 [[1263    2]
  [  13   33]]

 [[1204   12]
  [  23   72]]]


In [35]:
with ignore_warnings(category=UndefinedMetricWarning):
    print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.73      0.95      0.83       294
         1.0       0.82      0.78      0.80       160
         2.0       0.86      0.80      0.83       199
         3.0       0.82      0.62      0.71        95
         4.0       0.87      0.88      0.87       221
         5.0       0.88      0.77      0.82        75
         6.0       0.92      0.78      0.84       126
         7.0       0.94      0.72      0.81        46
         8.0       0.86      0.76      0.80        95

    accuracy                           0.82      1311
   macro avg       0.85      0.79      0.81      1311
weighted avg       0.83      0.82      0.82      1311



K-Neighbors classifier on single-output dataset

In [38]:
k_range = list(range(1,50))
weight_options = ["uniform", "distance"]

param_grid = dict(n_neighbors = k_range, weights = weight_options)

knn = KNeighborsClassifier()

grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid.fit(x_train, np.ravel(y_train, order='C'))

print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

0.9045801526717557
{'n_neighbors': 37, 'weights': 'distance'}
KNeighborsClassifier(n_neighbors=37, weights='distance')


In [39]:
clf = grid.best_estimator_

y_pred = clf.predict(x_test)

print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

# Cross validation

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

accuracies = cross_val_score(clf, x_train, y_train, cv=cv, scoring="accuracy", n_jobs=-1)
acc_svm = np.mean(accuracies)
print('Cross-validation score: ', acc_svm)

from sklearn.metrics import multilabel_confusion_matrix

# Confusion matrix

cm = multilabel_confusion_matrix(y_test, y_pred)

print('Confusion matrix: \n', cm)

Accuracy:  0.9328756674294432
Cross-validation score:  0.9065521628498726
Confusion matrix: 
 [[[ 985   32]
  [   6  288]]

 [[1138   13]
  [  14  146]]

 [[1101   11]
  [  14  185]]

 [[1210    6]
  [  20   75]]

 [[1080   10]
  [  10  211]]

 [[1232    4]
  [   7   68]]

 [[1180    5]
  [   6  120]]

 [[1261    4]
  [   1   45]]

 [[1213    3]
  [  10   85]]]


In [40]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.90      0.98      0.94       294
         1.0       0.92      0.91      0.92       160
         2.0       0.94      0.93      0.94       199
         3.0       0.93      0.79      0.85        95
         4.0       0.95      0.95      0.95       221
         5.0       0.94      0.91      0.93        75
         6.0       0.96      0.95      0.96       126
         7.0       0.92      0.98      0.95        46
         8.0       0.97      0.89      0.93        95

    accuracy                           0.93      1311
   macro avg       0.94      0.92      0.93      1311
weighted avg       0.93      0.93      0.93      1311



Multi-Layer Perceptron classifier on single-output dataset

In [6]:
# Pre-determined params
clf = MLPClassifier(hidden_layer_sizes = (128, 64, 32,), activation='relu', solver='adam', max_iter=1000)
with ignore_warnings(category=ConvergenceWarning):
    clf.fit(x_train, np.ravel(y_train, order='C'))

y_pred = clf.predict(x_test)

print('Accuracy: ', metrics.accuracy_score(y_test, y_pred))

# Cross validation

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

accuracies = cross_val_score(clf, x_train, y_train, cv=cv, scoring="accuracy", n_jobs=-1)
acc_svm = np.mean(accuracies)
print('Cross-validation score: ', acc_svm)

from sklearn.metrics import multilabel_confusion_matrix

# Confusion matrix

cm = multilabel_confusion_matrix(y_test, y_pred)

print('Confusion matrix: \n', cm)

Accuracy:  0.9221967963386728
Cross-validation score:  0.924554707379135
Confusion matrix: 
 [[[ 985   19]
  [  19  288]]

 [[1130   10]
  [  12  159]]

 [[1095   16]
  [  17  183]]

 [[1201   13]
  [   9   88]]

 [[1088   16]
  [  20  187]]

 [[1232    9]
  [   4   66]]

 [[1188   10]
  [  13  100]]

 [[1261    2]
  [   2   46]]

 [[1206    7]
  [   6   92]]]


In [7]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94       307
         1.0       0.94      0.93      0.94       171
         2.0       0.92      0.92      0.92       200
         3.0       0.87      0.91      0.89        97
         4.0       0.92      0.90      0.91       207
         5.0       0.88      0.94      0.91        70
         6.0       0.91      0.88      0.90       113
         7.0       0.96      0.96      0.96        48
         8.0       0.93      0.94      0.93        98

    accuracy                           0.92      1311
   macro avg       0.92      0.92      0.92      1311
weighted avg       0.92      0.92      0.92      1311



Load multi-output dataset

In [8]:
multi_data = np.loadtxt('tictac_multi.txt')
input_features = multi_data[:,:9]
output_labels = multi_data[:,9:]

input_features

array([[ 1., -1.,  0., ...,  0.,  1.,  0.],
       [ 1.,  0.,  1., ...,  0., -1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [-1., -1.,  0., ...,  1.,  1.,  0.],
       [ 0.,  1.,  1., ...,  0.,  1., -1.],
       [ 1.,  1., -1., ..., -1., -1.,  0.]])

Train-test split

In [9]:
train, test = train_test_split(multi_data, test_size=0.20)

x_train = train[:,:9]
x_test = test[:,:9]
y_train = train[:,9:]
y_test = test[:,9:]

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5240, 9)
(1311, 9)
(5240, 9)
(1311, 9)


Custom accuracy function (top prediction that hasn't been played is correct)

In [10]:
def check_accuracy(x_tests, y_tests, preds):
    total = len(y_tests)
    count_correct = 0.0

    for i in range(total):
        #sort the predicted values from greatest to least
        pred = preds[i]
        pred_sorted = np.sort(pred)
        pred_sorted = pred_sorted[::-1]

        # current x test board
        x_t = x_tests[i]

        # find highest confidence value not already
        # played on the board
        index = -1
        for j in range(len(pred_sorted)):
            # get index of highest confidence value
            l = np.where(pred == pred_sorted[j])
            # if that index is not played
            #print(x_t[l[0][0]])
            if int(x_t[l[0][0]]) == 0:
                index = int(l[0][0])
                break

        #best play
        #bp = [0]*9
        #bp[index] = 1
        #print("best play: ", bp)

        # current y (correct) board of possible plays
        y_t = y_tests[i]
        #print("yt: ", y_t)
        
        if y_t[index] == 1: count_correct += 1
            
    
    return float(count_correct) / float(total)

K-Nearest Neighbors on Dataset

In [12]:
reg = KNeighborsRegressor()
reg.fit(x_train, y_train)

y_pred = reg.predict(x_test)

print("Unaltered predictions\n")
print(y_pred[0:15])

# Round output values to either 0 or 1
y_pred_rounded = np.around(y_pred, decimals=0)
y_pred_rounded = np.clip(y_pred_rounded, 0, 1)
print("\nRounded predictions\n")
print(y_pred_rounded[0:15])

print("\nActual values\n")
print(y_test[0:15])

Unaltered predictions

[[1.  0.  0.4 0.4 0.6 0.  0.  0.  0. ]
 [0.2 0.2 0.6 0.2 0.2 0.2 0.4 0.2 0. ]
 [0.  0.  0.  0.  1.  0.  0.  0.  0. ]
 [0.4 0.  0.  0.4 0.2 0.2 0.  0.  0.2]
 [0.2 0.  0.  0.  0.8 0.  0.  1.  0.6]
 [0.  0.6 0.  0.  0.8 0.  0.  0.  0.4]
 [0.8 0.  0.  0.4 0.  0.  0.2 0.4 0.6]
 [0.8 0.  0.  0.  0.  0.  0.  0.  0.2]
 [0.  0.  0.  0.  0.2 0.  1.  0.  0.2]
 [1.  0.  0.  0.  0.  0.  0.  0.4 0. ]
 [0.  0.  0.  1.  0.6 0.4 0.8 0.4 0.6]
 [0.2 0.  0.2 0.8 0.  0.  0.  0.  0.2]
 [0.  0.2 0.  0.  0.  0.  0.  0.4 0.8]
 [0.2 0.  0.6 0.  0.8 0.4 0.  0.  0. ]
 [0.  0.  1.  0.  0.  0.  0.  0.  0. ]]

Rounded predictions

[[1. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 1.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 1. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0

Accuracy score (default)

In [13]:
print(metrics.accuracy_score(y_test, y_pred_rounded))

0.5461479786422578


Cross-validation

In [30]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

accuracies = cross_val_score(reg, x_train, y_train, cv=cv, n_jobs=-1)
acc_knn = np.mean(accuracies)
acc_knn

0.5508513732978059

The accuracy is low because the metric is comparing predictions for every tile on the board. However, in a practical setting, only the prediction with the highest confidence will be used, so we can just compare that one prediction.

Custom accuracy check

In [28]:
print(check_accuracy(x_test, y_test, y_pred))

0.9145690312738368


Support Vector Regressor on multi-output data

In [39]:
reg = MultiOutputRegressor(SVR())
reg.fit(x_train, y_train)

y_pred = reg.predict(x_test)

check_accuracy(x_test, y_test, y_pred)

0.9717772692601068

MLP Regressor on multi-output data

In [11]:
reg = MLPRegressor(hidden_layer_sizes=(128,64,32,), max_iter=1000)
reg.fit(x_train, y_train)

y_pred = reg.predict(x_test)

check_accuracy(x_test, y_test, y_pred)

0.9832189168573608

This accuracy should be good enough. We could optimize the parameters further, but this should be fine for the intended use. This is the model (and params) that we'll use.